# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/root/miniconda3/envs/peft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.50it/s]


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图之一，其符号是天。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、雨、露等自然现象。乾卦的意义包括力量、刚强、积极、行动、创造等。在八卦中，乾卦是最为积极主动和强大的卦象之一。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

周易中的讼卦（問證）是由两个阴爻夹一个阳爻构成，象征天下的判别。这个卦象主要表达了一种的需求和愿望，以及对于公正和公平的追求。

讼卦的意义包括解决争端、明辨是非、维护公正、伸张正义等。它告诉我们，在面对争议和冲突时，要寻求公正的评判，以达到和谐和平的状态。此外，讼卦还告诫我们要有耐心和毅力，对待争议和冲突要有正确的态度和方法。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240730_210817"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦，是周易中的一卦，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240730_210817）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240730_210817）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是由坎卦（水）和坤卦（地）相叠而成。在周易中，它代表着军队的胜利和胜利后的平安。这一卦象预示着吉祥吉祥，预示着吉祥的委托，吉祥的远处，吉利。师卦的核心哲学是：君子观此卦象，从而train卦象，训练自己的品德。师卦象数为3卦，卦辞为：颐德养性，涵真腹中，观此卦象，train卦象，train自己的品德。signifies the end of a cycle and the beginning of a new era.


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240730_210817）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。其卦辞为“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。"

在传统解卦中，乾卦预示着大吉大利，事业如日中天，但也提醒要警惕盛极必衰的道理。经商方面顺利发展，但要冷静分析形势，坚持商业道德。对于婚恋，尽管阳盛阴衰，但刚柔可相济，最终形成美满结果。总体而言，乾卦代表着充满活力和力量的时机，但也需要保持谦逊谨慎的态度，以应对可能出现的困难。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 
summary: "乾卦"

"乾卦" is the first hexagram in the I Ching, or the Book of Changes. In the traditional interpretation, it represents the天，即宇宙的运行和力量。乾卦象征 domains, 王权, 刚健, 强盛, 威武, 统治下的国家


乾卦由六个阳爻组成，象征着刚健的力量。它的卦辞是“元、亨、利、贞”，意味着它具有完全的道德品质。


在周易学中，乾卦代表着天、父亲、君主、刚强、强盛、威武、权威，以及它们之间的相互关系。人们普遍认为乾卦象征着世界运行的力量和规律，也象征着人们应该刚健不挠，追求刚健、强盛的精神。


在解卦中，乾卦表示刚强，阳盛，积极进取，追求卓越，其结果总是好的。在运势方面，乾卦能够带来权力、刚健、强盛、威武、繁荣等祝福。


在预测学中，乾卦预示着顺利、安全、强盛、繁荣，但也需要警惕潜在的困难和危险。


总结起来，乾卦是象征刚健、强盛、强硬、力量和目标的卦象，需要人们积极进取，追求卓越，同时也需要谨慎、稳定、和谐。"


In [16]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是一个由上卦坎（水）和下卦震（地）组成的卦象，代表着可预测的吉祥之象。在周易中，卦象被解释为天地间万物运行的模型，因此地水师卦被解释为地球上充满了水，而震卦表示地下面有水，下面又有地，水又依赖地而生存。

在卜筮时，地水师卦预示着辩明是非，伸张正义，能够成功。对于个体来说，这意味着在生活和工作中的困难和挫折可能会带来成长和改变。对于团队来说，这表示需要相互协作，互相支持，共同克服困难。

地水师卦的哲学意义在于，天地间万物相互依存，依赖对方的力量，双方合作可以成就大业。在个人生活中，面对困难和挫折时，应该勇于面对，积极寻求解决办法，依靠自己的智慧和毅力，克服困难。在团队中，成员之间应该相互协作，互相信任，共同实现目标。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由上卦坎（水）和下卦震（地）组成的卦象，上卦为坎，下卦为震，上卦为坎，下卦为震，象征着可变通达。

在卜筮时，可变通达之象预示着祥瑞，表示当事人将会遇到变化和进步。这个卦象可变通达，表示改变和变通是可能的。

在古代，地水师卦被认为是一种吉祥的征兆，预示着当事人将会顺利通过困难，实现愿望。

在现代，地水师卦仍然被认为是一种吉祥的征兆，象征着可变通达，变化和变通是可能的。

总结起来，地水师卦象征着变化和变通是可能的，预示着当事人将会顺利通过困难，实现愿望。


In [17]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，代表诉讼之卦，对应于春秋时期鲁国发生的一起诉讼事件。卦象为乾上坎下，意味着阳刚之健，阴柔之盛，象征刚健的君子在险恶的环境中保持冷静和理智。

讼卦的核心哲学是：矛盾双方均希望和要求美好，但由于双方的力量对比，难以达成共识，从而导致矛盾升级，最终不得不通过诉讼来解决。

讼卦的卦象 Like
像
像

解卦

初六：云雷声隐，尚可Watch

六二：ye舒中直，尚可喜

六三：舒中而反观，尚可忧

六四：反而振翅，尚不可测

六五：天降尤物，尚不可喜

六六：订阅尤物，不可敌

像
像

解卦

初六：像舒展开来的样子，尽管矛盾重重，但可以保持冷静，观察形势的发展，避免过于冲动。

六二：像舒展的手掌，矛盾已经明朗，应该积极寻求解决办法，避免让矛盾继续扩大。

六三：像舒展的手掌反顾，已有危机意识，应该及时调整策略，避免恶化局势。

六四：像鸟儿反顾，飞翔腾越，矛盾的解决需要耐心和智慧，不能急功近利。

六五：像天降下尤物，虽然令人担忧，但也有可能带来机会，需要密切关注形势变化。

六六：像订阅尤物，面临巨大压力，必须谨慎处理，避免陷入诉讼之灾。

 like
像

解卦

初六：像舒展开来的样子，尽管矛盾重重，但可以保持冷静，观察形势的发展，避免过于冲动。

六二：像舒展的手掌，矛盾已经明朗，应该积极寻求解决办法，避免让矛盾继续扩大。

六三：像舒展的手掌反顾，已有危机意识，应该及时调整策略，避免恶化局势。

六四：像鸟儿反顾，飞翔腾越，矛盾的解决需要耐心和智慧，不能急功近利。

六五：像天降下尤物，虽然令人担忧，但也有可能带来机会，需要密切关注形势变化。

六六：像订阅尤物，面临巨大压力，必须谨慎处理，避免陷入诉讼之灾。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦象

 在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [18]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [19]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的第一卦，卦象由六个阳爻（即阳性的木）排列而成，它们从下至上排列，看起来像是天上的云彩，因此得名“乾”。乾卦象征着刚强、坚毅和独立，以及与天地之间的联系。

乾卦的卦辞是：“乾为天，为父，为君，亲阳人。亲，先祖，家远的代际，宜先祖；国，先王，宜治国；夫，配偶，宜婚配；父，尊亲，宜尊父；君，朋友，宜友道。”卦辞描述了乾卦的内涵，它既象征着天，也象征着人。作为天，它与“先祖”和“先王”有关，表示宇宙的起点和万物的基础。作为人，它与“父”和“君”有关，表示人类的社会性和文化传承。

在《易经》中，乾卦是“八卦”之一，它既象征着阳，也象征着“刚”和“强”。乾卦的卦象既象征着“天”，也象征着“人”，它是“八卦”中最为重要的卦之一，表示宇宙的运行和万物的生成。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的第一卦，也是六十四卦中的首卦，它的卦象是由两个阳爻夹一个阴爻构成，象征着天，也象征着刚强和坚毅。乾卦的卦辞是“元，亨，利，贞”，意味着“ beginning, success, profit, integrity”。它表示万事万物的本源和原理，是成功的基础。乾卦 also means "the heaven", and it is a symbol of the universe, with six lines of yin and one line of yang. It represents the power and strength of the world, and is a symbol of integrity.


In [20]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦（师的卦象为乾为天，地为地）主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

师卦是由天（乾）和地（坤）组成的卦象，乾为天，坤为地。师卦是《易经》中第八卦，属于坤卦（ earth）系列卦象，坤卦下卦为坤，上卦为乾，乾为天，坤为地。

师卦象征着师兵，主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

师卦最佳状态为师的（师卦上卦为乾，下卦为坤），乾为天，坤为地，师卦象征师兵，主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

在占卦中，师卦卦象为吉凶相生，以阳刚为上，以顺为正，symbolism of the earth and sky，象征天地。师卦：师 师 师。三个师字，下坤上乾。

师卦卦象分析：师卦是乾卦下坤卦上，乾为天，坤为地，象征天和地。师卦象征师兵，主动向被动，主动而被动，地震师，吉。

师卦卦象占卜：占师卦，得师卦，吉。

师卦卦象应用：师卦象征师兵，主动向被动，主动而被动，象征天和地。适用于天和地的事物，占师卦 Creature and earth。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [21]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》中的第八卦，属于雷（卦头）坎（卦体）离（卦象）两种卦象的组合。讼卦象征诉讼、争议、争端等诉讼方面的现象。卦形为雷上坎下，雷象征刚毅，坎象征险，离象征烈。卦象下卦为坎，上卦为离，险险相间，刚毅相结合。

讼卦的卦辞：“讼，吉凶交战，得利藏损。”卦象下坎上离，坎险离光，光争下，险林光，刚毅争利，光损藏吉。

蔽卦（卦名：蔽）是《易经》中的第六卦，属于离（卦头）坤（卦体）震（卦象）两种卦象的组合。蔽卦象征屏蔽、遮蔽，阻止邪恶势力，弘扬正道。

蔽卦的卦辞：“蔽，利ulo，吉凶凶险皆屏蔽。”卦象下离上坤，离火遮蔽，坤地顺从，刚毅相合，和顺利害，吉祥安全。

 src="http://www.qk6.com/article/html/277776.html" target="_blank">http://www.qk6.com/article/html/277776.html


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
